# 04 Composition Explorer

**Purpose**: Visualize how bundle composition merge rules work.

This is a PLAYGROUND, not a tutorial. For learning, see:
- `examples/02_composition.py` - How composition works
- `docs/CONCEPTS.md` - Why composition exists

---

In [ ]:
# Setup - run this cell first

from amplifier_foundation import Bundle

## Define Base Bundle

This is your starting configuration.

In [ ]:
# Edit this base bundle
base = Bundle(
    name="base",
    version="1.0.0",
    session={
        "orchestrator": {"module": "loop-basic"},
        "context": {"module": "context-simple", "config": {"max_tokens": 100000}},
    },
    providers=[
        {"module": "provider-mock", "config": {"debug": False, "delay": 0}},
    ],
    tools=[
        {"module": "tool-filesystem"},
        {"module": "tool-bash"},
    ],
    instruction="Base instructions.",
)

print("BASE BUNDLE:")
print(f"  session: {base.session}")
print(f"  providers: {base.providers}")
print(f"  tools: {[t.get('module') for t in base.tools]}")
print(f"  instruction: '{base.instruction}'")

## Define Overlay Bundle

This is what gets merged ON TOP of the base.

In [ ]:
# Edit this overlay bundle
overlay = Bundle(
    name="overlay",
    version="1.0.0",
    session={
        # Only specifying context.config - orchestrator will be preserved
        "context": {"config": {"max_tokens": 200000, "auto_compact": True}},
    },
    providers=[
        # Same module ID = UPDATE existing config
        {"module": "provider-mock", "config": {"debug": True}},
        # New module ID = ADD to list
        {"module": "provider-anthropic", "config": {"default_model": "claude-sonnet-4-5"}},
    ],
    tools=[
        # New tool = ADD (existing tools preserved)
        {"module": "tool-web"},
    ],
    instruction="Overlay instructions.",  # REPLACES base instruction
)

print("OVERLAY BUNDLE:")
print(f"  session: {overlay.session}")
print(f"  providers: {overlay.providers}")
print(f"  tools: {[t.get('module') for t in overlay.tools]}")
print(f"  instruction: '{overlay.instruction}'")

## Compose and See Results

In [ ]:
# Compose: base + overlay
result = base.compose(overlay)

print("COMPOSED RESULT:")
print("=" * 50)

In [ ]:
# Session (DEEP MERGE)
print("\n1. SESSION (deep merge):")
print(f"   orchestrator: {result.session.get('orchestrator')}")
print(f"   context: {result.session.get('context')}")
print("\n   Note: orchestrator preserved from base, context.config merged")

In [ ]:
# Providers (MERGE BY MODULE ID)
print("\n2. PROVIDERS (merge by module ID):")
for p in result.providers:
    print(f"   - {p.get('module')}: {p.get('config', {})}")
print("\n   Note: provider-mock config merged (debug=True), provider-anthropic added")

In [ ]:
# Tools (MERGE BY MODULE ID)
print("\n3. TOOLS (merge by module ID):")
for t in result.tools:
    print(f"   - {t.get('module')}")
print("\n   Note: tool-web added, filesystem and bash preserved")

In [ ]:
# Instruction (REPLACE)
print("\n4. INSTRUCTION (replace):")
print(f"   '{result.instruction}'")
print("\n   Note: Overlay instruction completely replaced base instruction")

## Merge Rules Reference

| Section | Rule | Behavior |
|---------|------|----------|
| `session` | DEEP MERGE | Nested dicts merged recursively |
| `providers` | MERGE BY MODULE | Same module ID = update config, new = add |
| `tools` | MERGE BY MODULE | Same module ID = update config, new = add |
| `hooks` | MERGE BY MODULE | Same module ID = update config, new = add |
| `instruction` | REPLACE | Overlay completely replaces base |
| `includes` | RESOLVED FIRST | Not merged, resolved before composition |

---

## Experiment!

Try modifying the base and overlay bundles above to see how different changes affect the result.

Ideas to try:
- What happens if overlay has same tool as base?
- What if overlay doesn't set instruction?
- How does nested session config merge?
- What if you compose three bundles? (`base.compose(overlay1, overlay2)`)

In [ ]:
# Try composing multiple overlays
overlay2 = Bundle(
    name="overlay2",
    version="1.0.0",
    tools=[{"module": "tool-search"}],
)

multi_result = base.compose(overlay, overlay2)
print("Tools after base + overlay + overlay2:")
print([t.get("module") for t in multi_result.tools])